In [2]:
!pip install fastapi==0.75.0 uvicorn==0.17.6

In [3]:
!python -m pip install sqlalchemy==1.4.32

In [4]:
!python -m pip install python-dotenv==0.19.2

In [5]:
!python -m pip install validators==0.18.2

In [7]:
# Define Environment Variables
from functools import lru_cache
from pydantic import BaseSettings

class Settings(BaseSettings):
    env_name: str = "Local"
    base_url: str = "http://localhost:8000"
    db_url: str = "sqlite:///./shortener.db"

@lru_cache
def get_settings() -> Settings:
    settings = Settings()
    print(f"Loading settings for: {settings.env_name}")
    return settings

In [8]:
## Create  FastAPI App

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return "Welcome to the URL shortener API :)"

#### To launch  Swagger UI 
- check it out in your browser at http://127.0.0.1:8000/docs 

In [9]:
from pydantic import BaseModel

class URLBase(BaseModel):
    target_url: str

class URL(URLBase):
    is_active: bool
    clicks: int

    class Config:
        orm_mode = True

class URLInfo(URL):
    url: str
    admin_url: str

In [12]:
import validators
from fastapi import FastAPI, HTTPException

def raise_bad_request(message):
    raise HTTPException(status_code=400, detail=message)

@app.post("/url")
def create_url(url: URLBase):
    if not validators.url(url.target_url):
        raise_bad_request(message="Your provided URL is not valid")
    return f"TODO: Create database entry for: {url.target_url}"

In [13]:
##Prepare Your SQLite Database
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine(
    get_settings().db_url, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(
    autocommit=False, autoflush=False, bind=engine
)
Base = declarative_base()

Loading settings for: Development


In [14]:
# specifying how our data should be stored in the database
from sqlalchemy import Boolean, Column, Integer, String

class URL(Base):
    __tablename__ = "urls"

    id = Column(Integer, primary_key=True)
    key = Column(String, unique=True, index=True)
    secret_key = Column(String, unique=True, index=True)
    target_url = Column(String, index=True)
    is_active = Column(Boolean, default=True)
    clicks = Column(Integer, default=0)